In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pickle
import os
import json
import numpy as np


Firebase signin

In [2]:

cred = credentials.Certificate("../codeinterface-85b5e-firebase-adminsdk-11q7e-837ba92a03.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
docs = db.collection('tasks').get()
# save docs in a pickle file called old_tasks.pkl
tasks = {}
for doc in docs:
    doc_dict = doc.to_dict()
    doc_dict['id'] = doc.id 
    tasks[doc.id] = doc_dict


In [9]:
today_date_string = str(np.datetime64('now'))
with open('saved_data/old_tasks'+today_date_string+'.pkl', 'wb') as f:
    pickle.dump(tasks, f)

In [10]:
# remove all the documents from the collection
for doc in docs:
    doc.reference.delete()

In [11]:

folder_path = './tasks'  # Replace with the actual folder path

# Initialize a list to store all tasks
all_tasks = []

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as file:
            # Load JSON content
            task_data = json.load(file)
            # Add the task to the list
            all_tasks.append(task_data)

all_tasks = np.array(all_tasks)

In [13]:
# for each task in all_tasks print name and type
for task in all_tasks:
    print(task['name'], "\\\\ Type:  ", task['type'])

table_transform_unnamed2 \\ Type:   data_manip
retriever \\ Type:   lengthy_code
tokenizer \\ Type:   edit_code
is_bored \\ Type:   logic
t_test \\ Type:   data_manip_math
event_scheduler \\ Type:   logic
order_by_points \\ Type:   logic
encode_message \\ Type:   logic
triple_sum_to_zero \\ Type:   logic
even_odd_count \\ Type:   logic
sum_product \\ Type:   logic
calculator \\ Type:   fix_code
table_transform_unnamed1 \\ Type:   data_manip
login_authenticator \\ Type:   lengthy_code
is_multiply_prime \\ Type:   logic
count_nums \\ Type:   logic
table_transform_named \\ Type:   data_manip


In [14]:
# find task in all_tasks with field name  is Average
def get_task_index(task_name):
    for i in range(len(all_tasks)):
        if all_tasks[i]['name'] == task_name:
            return i
    # raise error if task not found
    raise ValueError('Task not found')
tutorial_task_index = get_task_index('sum_product')
tasks_chosen = np.array([ get_task_index('count_nums'), get_task_index('t_test'),   get_task_index('table_transform_unnamed1'), get_task_index('login_authenticator'),
                          get_task_index('retriever'), get_task_index('calculator'), get_task_index('is_bored')])

tasks_chosen = np.array(range(len(all_tasks)))

# 2 logic problems, 1 data manipulation or lengthy code or edit code, logic and fill the rest 
# 5 sets of tasks - first 15mins should be the same 


How many sets of tasks and how do we configure each set

In [15]:
all_tasks[tasks_chosen]

array([{'name': 'table_transform_unnamed2', 'task_description': '\n+----+--------+--------+---------+--------+--------+\n|    |   col1 |   col2 |    col3 |   col4 |   col5 |\n|----+--------+--------+---------+--------+--------|\n|  0 |      6 |      1 | 5.38817 |      3 |      2 |\n|  1 |      9 |      2 | 4.19195 |      5 |      8 |\n|  2 |     10 |      8 | 6.8522  |      8 |      1 |\n|  3 |      6 |      7 | 2.04452 |      8 |      7 |\n|  4 |      1 |     10 | 8.78117 |     10 |     10 |\n+----+--------+--------+---------+--------+--------+\n\nThis is a pandas dataframe provided to you above as input.\n\nYou need to transform it exactly to the following output dataframe by recognizing the relationship between the input and output dataframes.\n\n+----+--------+--------+----------+\n|    |   col1 |   col2 |     col3 |\n|----+--------+--------+----------|\n|  0 |      6 |      2 |  8.38817 |\n|  1 |     15 |      3 |  9.19195 |\n|  2 |     25 |      9 | 14.8522  |\n|  3 |     31 |   

In [16]:

# Extracting data for the tutorial task
tutorial_task = all_tasks[tutorial_task_index]

selected_tasks = [all_tasks[i] for i in tasks_chosen]

# Creating the dictionary with the specified structure
fb_taskgpt = {
    'function_signatures': [task['function_signature'] for task in selected_tasks],
    'task_descriptions': [task['task_description'] for task in selected_tasks],
    'tutorial_function_signature': tutorial_task['function_signature'],
    'model': "gpt-3.5-turbo",  # As specified, for no model it is None
    'tutorial_task_description': tutorial_task['task_description'],
    'unit_tests': [task['unit_test'] for task in selected_tasks],
    'exp_condition': 'pilotchat',  # Placeholder, as the actual condition is not specified
    'max_tokens': 150,  # Placeholder, adjust as necessary
    'tutorial_unit_test': tutorial_task['unit_test'],
    'id': "pilot_gpt35_chat",
    'task_completed': 0,
    'interface_type': 'chat'
}

fb_taskllama = {
    'function_signatures': [task['function_signature'] for task in selected_tasks],
    'task_descriptions': [task['task_description'] for task in selected_tasks],
    'tutorial_function_signature': tutorial_task['function_signature'],
    'model': "CodeLlama-7b-Instruct",  # As specified, for no model it is None
    'tutorial_task_description': tutorial_task['task_description'],
    'unit_tests': [task['unit_test'] for task in selected_tasks],
    'exp_condition': 'pilotchat',  
    'max_tokens': 150,  
    'tutorial_unit_test': tutorial_task['unit_test'],
    'id': "pilot_llama7_chat",
    'task_completed': 0,
    'interface_type': 'chat'

}


for i in range(10):
    doc_ref = db.collection(u'tasks').document(u'pilot_gpt35chat'+str(i))
    doc_ref.set(fb_taskgpt)

    doc_ref = db.collection(u'tasks').document(u'pilot_llama7chat'+str(i))
    doc_ref.set(fb_taskllama)


